In [ ]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

# 適宜自分のApplicationPATHまで変更
APP_PATH = '/content/drive/Othercomputers/LAPTOP-3BHDM0TI/windows_research/workspace/research/carbon-steel-classification'


import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch.nn.functional as F
from torch import optim

from sklearn.model_selection import train_test_split

import os
import sys
import cv2
from PIL import Image
import pandas as pd


os.chdir(APP_PATH)
!pwd
!ls -al

Mounted at /content/drive
/content/drive/Othercomputers/LAPTOP-3BHDM0TI/windows_research/workspace/research/carbon-steel-classification
total 31
drwx------ 2 root root 4096 May  8 09:43 app
drwx------ 2 root root 4096 May  8 09:52 bin
drwx------ 2 root root 4096 May  9 20:26 data
drwx------ 2 root root 4096 May 10 06:28 dataset_npz
drwx------ 2 root root 4096 May  8 09:16 .git
-rw------- 1 root root   48 May  8 09:58 .gitignore
-rw------- 1 root root  265 May  8 09:51 Makefile
-rw------- 1 root root  358 May  8 09:37 README.md
drwx------ 2 root root 4096 May  8 09:38 tmp
-rw------- 1 root root 4773 May 19 10:55 TODO.txt


In [ ]:
datasets_npz_x1 = np.load(APP_PATH + '/dataset_npz/x.1_dataset/sc_x10_960_x1.npz')

CATEGORIES=["S10C","S15C","S25C","S35C","S45C","S55C"]



images = datasets_npz_x1['x']
labels = datasets_npz_x1['y']

images_list = []
labels_list = []

NUM_DATA = 10

for i, image in enumerate(images):
    # CPUが使い果たされてしまうので200までを使用
    if i == NUM_DATA:
        break

    # 標準化もここで行なう？？
    image = Image.fromarray(image)
    # print("image dtype: {}".format(type(image)))
    image = image.convert("RGB")
    # print("image mode: {}".format(image.mode)) # むりやり3チャンネルにしているので表記上はRGBだが実際はL
    image = np.asarray(image, np.uint8)
    # print("image shape: {}".format(image.shape))
    image = np.transpose(image, (2, 0, 1))
    print("image transpose shape: {}".format(image.shape))
    # transpose_images = np.append(transpose_images, image, axis=0)
    images_list.append(image)
    labels_list.append(labels[i])
    
    print(i, '/', len(images))


transpose_images = np.array(images_list)
transpose_labels = np.array(labels_list)

# うまくいっていたら保存
print("transpose_images dtype: {}".format(type(transpose_images)))
print("transpose_images shape: {}".format(transpose_images.shape))
print("transpose_images: {}".format(transpose_images))

print("transpose_labels dtype: {}".format(type(transpose_labels)))
print("transpose_labels shape: {}".format(transpose_labels.shape))
print("transpose_labels: {}".format(transpose_labels))
np.savez(APP_PATH, 'data/np_savez', transpose_images, datasets_npz_x1['y'])

image transpose shape: (3, 960, 960)
0 / 290
image transpose shape: (3, 960, 960)
1 / 290
image transpose shape: (3, 960, 960)
2 / 290
image transpose shape: (3, 960, 960)
3 / 290
image transpose shape: (3, 960, 960)
4 / 290
image transpose shape: (3, 960, 960)
5 / 290
image transpose shape: (3, 960, 960)
6 / 290
image transpose shape: (3, 960, 960)
7 / 290
image transpose shape: (3, 960, 960)
8 / 290
image transpose shape: (3, 960, 960)
9 / 290
transpose_images dtype: <class 'numpy.ndarray'>
transpose_images shape: (10, 3, 960, 960)
transpose_images: [[[[ 65  54  47 ... 117 119 115]
   [ 74  62  53 ... 110 115 107]
   [ 74  88 112 ... 115 113 111]
   ...
   [ 80  79  89 ...  90  98 134]
   [ 84  82 100 ... 239 240 235]
   [ 85  84 103 ... 238 236 236]]

  [[ 65  54  47 ... 117 119 115]
   [ 74  62  53 ... 110 115 107]
   [ 74  88 112 ... 115 113 111]
   ...
   [ 80  79  89 ...  90  98 134]
   [ 84  82 100 ... 239 240 235]
   [ 85  84 103 ... 238 236 236]]

  [[ 65  54  47 ... 117 119 

In [ ]:
# transformsなしの自作のデータセット作成

data = torch.tensor(transpose_images, dtype=torch.float32)
label = torch.tensor(transpose_labels, dtype=torch.int64)

print('data.shape:', data.shape, 'label.shape:', label.shape)

# Datasetを作成
dataset = torch.utils.data.TensorDataset(data, label)

X_sample, y_sample = dataset[0]
print(X_sample.shape, y_sample.shape)
print(X_sample)
print(y_sample)

data.shape: torch.Size([10, 3, 960, 960]) label.shape: torch.Size([10])
torch.Size([3, 960, 960]) torch.Size([])
tensor([[[ 65.,  54.,  47.,  ..., 117., 119., 115.],
         [ 74.,  62.,  53.,  ..., 110., 115., 107.],
         [ 74.,  88., 112.,  ..., 115., 113., 111.],
         ...,
         [ 80.,  79.,  89.,  ...,  90.,  98., 134.],
         [ 84.,  82., 100.,  ..., 239., 240., 235.],
         [ 85.,  84., 103.,  ..., 238., 236., 236.]],

        [[ 65.,  54.,  47.,  ..., 117., 119., 115.],
         [ 74.,  62.,  53.,  ..., 110., 115., 107.],
         [ 74.,  88., 112.,  ..., 115., 113., 111.],
         ...,
         [ 80.,  79.,  89.,  ...,  90.,  98., 134.],
         [ 84.,  82., 100.,  ..., 239., 240., 235.],
         [ 85.,  84., 103.,  ..., 238., 236., 236.]],

        [[ 65.,  54.,  47.,  ..., 117., 119., 115.],
         [ 74.,  62.,  53.,  ..., 110., 115., 107.],
         [ 74.,  88., 112.,  ..., 115., 113., 111.],
         ...,
         [ 80.,  79.,  89.,  ...,  90.,  98., 

In [ ]:
# DataLoader ミニバッチ学習
# 入力値と目標値をまとめる


# 学習データ、検証データ、テストデータに分ける
# 各データセットのサンプル数を決定
# train : val : test = 60% : 20% : 20%
num_train = int(len(dataset) * 0.8)
# num_val = int(len(dataset) * 0.2)
num_test = int(len(dataset) - num_train)

print('num_train:', num_train)
# print('num_val:', num_val)
print('num_test:', num_test)

# ランダムにするらめにシードの固定をした
torch.manual_seed(0)
# データセットの分割
train, test = torch.utils.data.random_split(dataset, [num_train, num_test])
print('train:', train)
# print('val:', val)
print('test:', test)

num_train: 8
num_test: 2
train: <torch.utils.data.dataset.Subset object at 0x7f99381ce770>
test: <torch.utils.data.dataset.Subset object at 0x7f99381ce8c0>


In [ ]:
# ミニバッチ学習

# バッチサイズの定義(目安は全データの1/10だが正解がない)
batch_size = 2
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
# val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

# 1つ目のバッチサイズに分割したデータを確認
x, t = next(iter(train_loader))

print('x:', x)
print('t:', t)

x: tensor([[[[169., 169., 169.,  ..., 178., 169., 181.],
          [171., 171., 171.,  ..., 177., 171., 180.],
          [174., 174., 174.,  ..., 174., 175., 177.],
          ...,
          [ 95.,  87.,  80.,  ..., 175., 174., 174.],
          [109.,  89.,  88.,  ..., 173., 172., 172.],
          [151., 104.,  85.,  ..., 171., 170., 170.]],

         [[169., 169., 169.,  ..., 178., 169., 181.],
          [171., 171., 171.,  ..., 177., 171., 180.],
          [174., 174., 174.,  ..., 174., 175., 177.],
          ...,
          [ 95.,  87.,  80.,  ..., 175., 174., 174.],
          [109.,  89.,  88.,  ..., 173., 172., 172.],
          [151., 104.,  85.,  ..., 171., 170., 170.]],

         [[169., 169., 169.,  ..., 178., 169., 181.],
          [171., 171., 171.,  ..., 177., 171., 180.],
          [174., 174., 174.,  ..., 174., 175., 177.],
          ...,
          [ 95.,  87.,  80.,  ..., 175., 174., 174.],
          [109.,  89.,  88.,  ..., 173., 172., 172.],
          [151., 104.,  85.,  

In [ ]:
# 2.3 ニューラルネットワークの定義

num_classes = 6    # CIFAR10のクラスの数を指定

n_input = 32 * 32 * 3 # 全結合するために使用する
n_output = 6
# 中間層のノード数(何でもよい：調べる必要あり)
n_hidden = 128

# https://www.youtube.com/watch?v=6roIEgXy7wA
class CNN(nn.Module):
    def __init__(self, n_output, n_hidden):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.maxpool = nn.MaxPool2d((2, 2))
        # self.flatten = nn.Flatten()
        # self.flatten = nn.Flatten()
        self.l1 = nn.Linear(3625216, n_hidden)
        self.l2 = nn.Linear(n_hidden, n_output)
        self.relu = nn.ReLU(inplace=True)

    # 順伝搬を定義
    def forward(self, x):
        x = self.maxpool(self.relu(self.conv1(x)))
        x = self.maxpool(self.relu(self.conv2(x)))
        # x = torch.flatten(x)
        x = torch.flatten(x)
        x = self.relu(self.l1(x))
        x = self.l2(x)
        return x

# flattenに入れる前の入力画像(特徴MAP)のサイズ確認
conv1 = nn.Conv2d(3, 32, 3)
conv2 = nn.Conv2d(32, 32, 3)
maxpool = nn.MaxPool2d((2, 2))
relu = nn.ReLU(inplace=True)

x = maxpool(relu(conv1(x)))
x = maxpool(relu(conv2(x)))
x = torch.flatten(x)

print("x shape: {}".format(x.shape))


# ネットワークのロード
# CPUとGPUのどちらを使うかを指定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# net = AlexNet().to(device)
print("Device: {}".format(device))
net = CNN(n_output, n_hidden).to(device)
print(net)

x shape: torch.Size([3625216])
Device: cuda
CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (l1): Linear(in_features=3625216, out_features=128, bias=True)
  (l2): Linear(in_features=128, out_features=6, bias=True)
  (relu): ReLU(inplace=True)
)


In [ ]:
# 2.4 損失関数と最適化関数の定義

# 損失関数の定義
criterion = nn.CrossEntropyLoss()

# 活性化関数の定義(最適化アルゴリズムの設定(確率的勾降下法))
lr = 0.01
# optimizer = optim.Adam(net.parameters())
optimizer = optim.SGD(net.parameters(), lr=lr)

history = np.zeros((0,5))
history

array([], shape=(0, 5), dtype=float64)

In [ ]:
# 2.5 学習

from tqdm.notebook import tqdm

# # 損失と正解率を保持するリストを作成
# train_loss_list = []        # 学習損失
# train_accuracy_list = []    # 学習データの正答率
# test_loss_list = []         # 評価損失
# test_accuracy_list = []     # テストデータの正答率

num_epoch = 10

for epoch in range(num_epoch):
    train_acc, train_loss = 0, 0
    val_acc, val_loss = 0, 0
    n_train, n_test = 0, 0

    for inputs, labels in tqdm(train_loader):
        
        print("inputs type: {}".format(type(inputs)))
        print("labels type: {}".format(type(labels)))
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = net(inputs)
        
        print("inputs shape: {}".format(inputs.shape))
        print("labels shape: {}".format(labels.shape))
        
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        predicted = torch.max(outputs, 1)[1]

        train_loss += loss.item()
        train_acc += (predicted == labels).sum().item()
    
    for inputs_test, labels_test in test_loader:
        n_test += len(labels_test)

        inputs_test = inputs_test.to(device)
        labels_test = labels_test.to(device)
        
        outputs_test = net(inputs_test)
        
        loss_test = criterion(outputs_test, labels_test)

        predicted_test = torch.max(outputs_test, 1)[1]
        
        val_loss += loss_test.item()
        val_acc += (predicted_test == labels_test).sum().item()


    train_acc = train_acc / n_train
    val_acc = val_acc / n_test
    train_loss = train_loss * batch_size / n_test
    val_loss = val_loss * batch_size / n_test
    
    print(f'Epoch [{epoch+1}/{num_epoch}], loss:{train_loss:.5f} acc:{train_acc:.5f} val_loss:{val_acc:.5f}, val_acc:{val_acc:.5f}')
    items = np.array([epoch+1, train_loss, train_acc, val_loss, val_acc])
    history = np.vstack((history.items))

  0%|          | 0/4 [00:00<?, ?it/s]

inputs type: <class 'torch.Tensor'>
labels type: <class 'torch.Tensor'>
inputs shape: torch.Size([2, 3, 960, 960])
labels shape: torch.Size([2])


RuntimeError: ignored

In [ ]:
# 2.6 結果の可視化

# 損失
plt.figure()
plt.title('Train and Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(range(1, epoch+1), train_loss_list, color='blue', linestyle='-', label='Train_Loss')
plt.plot(range(1, epoch+1), test_loss_list, color='red', linestyle='--', label='Test_Loss')
plt.legend()    # 凡例

# 正解率
plt.figure()
plt.title('Train and Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.plot(range(1, epoch+1), train_accuracy_list, color='blue', linestyle='-', label='Train_Accuracy')
plt.plot(range(1, epoch+1), test_accuracy_list, color='red', linestyle='--', label='Test_Accuracy')
plt.legend()    # 凡例

# 表示
plt.show()

In [ ]:
# 分類した画像を確認
# CIFAR10のクラス
# classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']  # CIFAR10のクラス
CATEGORIES=["S10C","S15C","S25C","S35C","S45C","S55C"]


# ニューラルネットワークを評価モードに設定
net.eval()
# 評価時の計算で自動微分機能をオフにする
with torch.no_grad():
    for images, labels in test_loader:
        # GPUにTensorを転送
        images = images.to(device)
        labels = labels.to(device)
        # データを入力して予測値を計算(順伝播)
        y_pred_prob = net(images)
        # 予測したラベルを予測確率y_pred_probから計算
        y_pred_labels = torch.max(y_pred_prob, 1)[1]
        
        for i in range(9):
            image = images[i] / 2 + 0.5
            image = image.to('cpu').numpy()
            plt.subplot(3, 3, i+1)
            # matplotlibでは(縦, 横, チャネル)の順
            plt.imshow(np.transpose(image, (1, 2, 0)))
            plt.title(CATEGORIES[labels[i]])
            plt.title("{} ({})".format(CATEGORIES[y_pred_labels[i].item()], CATEGORIES[labels[i].item()]), color=('green' if y_pred_labels[i] == labels[i] else 'red'))
            plt.axis('off')
        plt.show()
        break
